In [ ]:
import pandas as pd
# import numpy as np
# from datetime import time, datetime
# from matplotlib import pyplot as plt
# from matplotlib.ticker import MultipleLocator
# import matplotlib.ticker as ticker
# from matplotlib.dates import DateFormatter
import os

In [ ]:
workout_type = 'A'

In [ ]:
FILENAME = 'raw_data/CSVs/erg scores - 2023-03-15.csv'
a = pd.read_csv(FILENAME)

In [ ]:
def split_to_seconds(duration_string: str):
    return float(duration_string.split(':')[0])*60+float(duration_string.split(':')[1])

a["split_in_s"] = a["/500m"].apply(lambda x: split_to_seconds(x))

In [ ]:
# from datetime import datetime, timedelta

# def convert_str_duration_to_time_delta(duration_string: str):
#     dt = datetime.strptime(duration_string, "%M:%S.%f")
#     return timedelta(minutes=dt.minute, seconds=dt.second, microseconds=dt.microsecond)

# a["/500m_duration"] = a["/500m"].apply(lambda x: convert_str_duration_to_time_delta(x))

In [ ]:
a['cutoff_minute'] = a['time'].str.split(':').str[0].astype(int)

In [ ]:
a['workout_date'] = FILENAME[-14:-4]
a['workout_type'] = workout_type
a['workout_date'] = pd.to_datetime(a['workout_date']).dt.tz_localize('Europe/London')

In [8]:
import pandera as pa
from pandera import Column, DataFrameSchema, Check

In [ ]:
allowed_workout_types = ['A', 'B']

schema = pa.DataFrameSchema({
    'workout_date': pa.Column(pa.engines.pandas_engine.DateTime(tz='Europe/London')),
    'workout_type': pa.Column(pa.Category, checks = pa.Check.isin(allowed_workout_types), coerce=True, regex = True),
    'cutoff_minute': pa.Column(int, coerce=True),
    'split_in_s': pa.Column(float, coerce=True),
    'meter': pa.Column(int, coerce=True)
})

In [ ]:
OUTPUT_FILEPATH = f'processed_data/workout_{workout_type}.csv'
if os.path.isfile(OUTPUT_FILEPATH):
    output = pd.read_csv(OUTPUT_FILEPATH)
    output = pd.concat([output, a], ignore_index=True)
    output.to_csv(OUTPUT_FILEPATH, index=False)
else:
    a.to_csv(OUTPUT_FILEPATH, index=False)

In [2]:

# date='2023-03-01'

import os

import boto3
import pandas as pd

boto3.setup_default_session(profile_name='dev-profile')
client = boto3.client("s3")
bucket = 'ergscorepipeline'

for key in client.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])

# result = client.get_bucket_policy(Bucket=bucket)
# print(result['Policy'])


raw-data/
raw-data/manual-input/
raw-data/manual-input/2022-03-28.csv
raw-data/manual-input/2022-04-18.csv
raw-data/manual-input/2022-05-30.csv
raw-data/manual-input/2022-06-13.csv
raw-data/manual-input/2022-06-27.csv
raw-data/manual-input/2022-07-18.csv
raw-data/manual-input/2022-07-25.csv
raw-data/manual-input/2022-09-06.csv
raw-data/manual-input/2022-10-17.csv
raw-data/manual-input/2022-10-24.csv
raw-data/manual-input/2022-10-31.csv
raw-data/manual-input/2022-11-07.csv
raw-data/manual-input/2022-11-15.csv
raw-data/manual-input/2022-11-21.csv
raw-data/manual-input/2022-11-29.csv


In [3]:
import io

In [4]:
bucket = 'ergscorepipeline'
key = 'raw-data/manual-input/2022-03-28.csv'

obj = client.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(io.BytesIO(obj['Body'].read()))

In [5]:
df

,cutoff_minute,metres,/500m,s/m
0,6,1223,2:27.1,20
1,12,1224,2:27.0,20
2,18,1221,2:27.4,20
3,24,1228,2:26.5,20
4,30,1245,2:24.5,20


In [9]:
cli_input_schema = pa.DataFrameSchema({
    'cutoff_minute': pa.Column(int, coerce=True),
    'metres': pa.Column(
        int, Check(lambda x: 1000 <= x <= 2000, element_wise=True,
                   error="range checker [1000, 2000]")),
    's/m': pa.Column(
        int, Check(lambda x: 18 <= x <= 23, element_wise=True,
                   error="range checker [18, 23]")),
    '/500m': pa.Column(str, Check.str_matches("\d\:[0-5]\d\.\d"))
})

In [10]:
cli_input_schema.validate(df)

,cutoff_minute,metres,/500m,s/m
0,6,1223,2:27.1,20
1,12,1224,2:27.0,20
2,18,1221,2:27.4,20
3,24,1228,2:26.5,20
4,30,1245,2:24.5,20


In [13]:
bucket = 'ergscorepipeline'
key = 'raw-data/manual-input/2022-03-28.csv'
date = '2022-03-28'
def split_to_seconds(duration_string: str):
    return float(duration_string.split(':')[0])*60+float(duration_string.split(':')[1])
def process(bucket, key, date):
    obj = client.get_object(Bucket=bucket, Key=key)
    erg_scores_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    erg_scores_df["split_in_s"] = erg_scores_df["/500m"].apply(lambda x: split_to_seconds(x))
    erg_scores_df['workout_date'] = pd.to_datetime(date).tz_localize('Europe/London')
    return erg_scores_df

process(bucket,key,date)

,cutoff_minute,metres,/500m,s/m,split_in_s,workout_date
0,6,1223,2:27.1,20,147.1,2022-03-28 00:00:00+01:00
1,12,1224,2:27.0,20,147.0,2022-03-28 00:00:00+01:00
2,18,1221,2:27.4,20,147.4,2022-03-28 00:00:00+01:00
3,24,1228,2:26.5,20,146.5,2022-03-28 00:00:00+01:00
4,30,1245,2:24.5,20,144.5,2022-03-28 00:00:00+01:00


In [14]:
processed_schema = pa.DataFrameSchema({
    'workout_date': pa.Column(pa.engines.pandas_engine.DateTime(tz='Europe/London')),
    'cutoff_minute': pa.Column(int, coerce=True),
    'split_in_s': pa.Column(float, coerce=True),
})

processed_schema.validate(process(bucket,key,date))

,cutoff_minute,metres,/500m,s/m,split_in_s,workout_date
0,6,1223,2:27.1,20,147.1,2022-03-28 00:00:00+01:00
1,12,1224,2:27.0,20,147.0,2022-03-28 00:00:00+01:00
2,18,1221,2:27.4,20,147.4,2022-03-28 00:00:00+01:00
3,24,1228,2:26.5,20,146.5,2022-03-28 00:00:00+01:00
4,30,1245,2:24.5,20,144.5,2022-03-28 00:00:00+01:00
